In [ ]:
from llama import llama_api
#from gpt4 import gpt4o_api
from score import comparison

In [ ]:
input_file = "example_input.txt"
reference_file = "example_truth.txt"


In [ ]:
# my_prompt = "Read the following document and write a summary that includes the background of the case in one paragraph, the procedural history in two paragraphs, and the court's decision and reasoning in one paragraph."
# print(my_prompt)

In [ ]:
# Summarize the most essential parts of the case
prompt = "State the background of the case in one paragraph in the following document."
facts = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_facts.txt")
print("Facts done.")

prompt = "Summarize the procedural history in two paragraphs in the following document."
history = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_history.txt")
print("history done.")

prompt = "State the court's decision and reasoning in one paragraph in the following document."
reasoning = llama_api(prompt=prompt, input_file=input_file, output_file="llama_agent_reasoning.txt")
print("reasoning done.")

# Get the more concise conclusions which are put at the front of the summary.
case_summary = "\n\n".join([facts, history, reasoning])

prompt = "In one simple clause, write a concise headline that summarizes the court's decision in the following summary:"
decision = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_decision.txt")
print("decision done.")

prompt = "In one simple sentence, write a concise legal statement that summarizes the central question in the following case summary:"
issue = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_issue.txt")
print("issue done.")

# Organize to form the final output summary.
combined_summary = "\n\n".join([decision, issue, facts, history, reasoning])

prompt = "Combine the following pieces of text into a single cohesive passage. Ensure the flow is smooth and logical while keeping most of the original content unchanged. Edit only as needed for readability and natural transitions"
final_output = llama_api(prompt=prompt, text=combined_summary, output_file="llama_final_output.txt")
print("combine done.")

In [ ]:
comparison(reference_file, "llama_final_output.txt", "final_comparison.txt")

## Combine Prompt

In [ ]:
agents_prompts = {
    #"agent_1": (
    #    "Generate a concise title summarizing the court's decision in this case. "
    #    "The title must begin with the name of the court with the summary of the court's decision."
    #),
    #"agent_2": (
    #    "Write an introductory sentence starting with 'This appeal'. "
    #    "Explain the central legal question in the case in one sentence."
    #),
    "agent_1": (
        "Summarize the background of the case in one concise paragraph. Include the complainant, the accused, the alleged offences, and references to applicable sections of the Criminal Code."
        "Focus on the complainant, the accused, and the core allegations."
    ),
    "agent_2": (
        "Summarize the entire court proceedings in one concise paragraph, including: "
        "- The trial process: Describe the evidence presented, the trial judge's decision, and their reasoning. "
        "- The appellate process: Highlight the appellate arguments, the appellate courts' reasoning, and their decisions. "
        "- The final decision: Summarize the final judgment and reasoning of the highest court involved in this case. "
        "Focus on the logical progression of the case and key reasoning at each stage."
    ),
    "agent_3": (
        "Summarize the most critical legal principle established in this case in one concise sentence. "
        "Focus on the general relevance of the evidence, its relationship to the legal elements of the offence, and ensure the principle can apply broadly in similar cases."
    ),
    "agent_4": (
        "Write a concluding paragraph starting with 'Writing for the'. "
        "Summarize the majority opinion, focusing on the reasoning and its implications for the case."
    ),
}


# save results
generated_paragraphs = {}

# generate texts
for agent_name, prompt in agents_prompts.items():
    output_file = f"{agent_name}_output.txt"
    print(f"Running {agent_name}...")
    content = llama_api(
        prompt=prompt,
        input_file=input_file,
        output_file=output_file
    )
    generated_paragraphs[agent_name] = content
    print(f"{agent_name} completed. Output saved to {output_file}")

agent_5_prompt = (
    "Combine the following text outputs from seven agents into a single cohesive and well-structured summary. "
    "Follow this structure and ensure each section adheres to the specified format: "
    "1. Title: Begin with the name of the court and provide a concise summary of the decision. For example, 'The Supreme Court sets aside...'. "
    "2. Introduction: Write one sentence starting with 'This appeal' that summarizes the central legal question. "
    "3. Background: Summarize the case facts, including the complainant, the accused, the alleged offences, and references to relevant legal provisions. "
    "4. Court Proceedings: Describe the entire court process in one paragraph, including the trial, appellate decisions, and final judgment, ensuring smooth progression. "
    "5. Legal Principle: Provide the most critical legal principle established by the court in one concise sentence. Ensure it is general and applicable to similar cases. "
    "6. Conclusion: Write a concluding paragraph starting with 'Writing for the', summarizing the court's reasoning and its broader implications for the law. "
    "- Remove all redundancy or repetition while preserving critical legal principles and facts. "
    "- Ensure smooth transitions between sections, with each part contributing to a clear and logical narrative."
    "- Part 5 should be absolutely one sentence, and not include the specific cases or facts."
    "- part 6 should not be changed too much."
)

# combine all paragraphs
all_paragraphs_combined = "\n\n".join(generated_paragraphs.values())

# final
final_output = llama_api(
    prompt=agent_5_prompt,
    text=all_paragraphs_combined,
    output_file="final_case_summary_integrated.txt"
)
print("Final integrated summary saved to final_case_summary_integrated.txt.")


In [ ]:
# with open("multi_agent_concat_results/llama_output_facts.txt", "r") as file:
#     facts = file.read()

# with open("multi_agent_concat_results/llama_output_history.txt", "r") as file:
#     history = file.read()

# with open("multi_agent_concat_results/llama_output_reasoning.txt", "r") as file:
#     reasoning = file.read()

In [ ]:
# # Get the more concise conclusions which are put at the front of the summary.
# case_summary = "\n\n".join([facts, history, reasoning])

# prompt = "In one simple clause, write a concise headline that summarizes the court's decision in the following summary:"
# decision = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_decision.txt")
# print("decision done.")

# prompt = "In one simple sentence, write a concise legal statement that summarizes the central question in the following case summary:"
# issue = llama_api(prompt=prompt, text=case_summary, output_file="llama_agent_issue.txt")
# print("issue done.")

# # Organize to form the final output summary.
# combined_summary = "\n\n".join([decision, issue, facts, history, reasoning])

# prompt = "Combine the following pieces of text into a single cohesive passage. Ensure the flow is smooth and logical while keeping most of the original content unchanged. Edit only as needed for readability and natural transitions"
# final_output = llama_api(prompt=prompt, text=combined_summary, output_file="llama_final_output.txt")
# print("combine done.")

In [ ]:
# gpt4_output_file = "gpt4_output.txt"
# gpt4_comparison_file = "gpt4_comparison.txt"

# gpt4o_api(my_prompt, input_file, gpt4_output_file)
# comparison(reference_file, gpt4_output_file, gpt4_comparison_file)

In [ ]:
# comparison(reference_file, "combined_output.txt", "combined_comparison.txt")